# Laboratorium 5 - rekomendacje grupowe

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas matplotlib`

## Część 1. - przygotowanie danych

In [6]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from random import choice, sample
from statistics import mean, stdev

from reco_utils import *

In [7]:
# wczytujemy oceny uytkownikow i obliczamy (za pomocą collaborative filtering) wszystkie przewidywane oceny filmow

raw_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
movies = list(raw_ratings['movieId'].unique())
users = list(raw_ratings['userId'].unique())
ratings = get_predicted_ratings(raw_ratings)
ratings

Total error: 214288.32789758118
Total error: 207430.17864485475
Total error: 201069.00142627288
Total error: 195152.70220662426
Total error: 189635.99298642896
Total error: 184479.34764583927
Total error: 179648.142071672
Total error: 175111.94155736687
Total error: 170843.90680859485
Total error: 166820.2961550259
Total error: 163020.04632158997
Total error: 159424.41774867979
Total error: 156016.6932559958
Total error: 152781.92102644293
Total error: 149706.69459622234
Total error: 146778.96388693393
Total error: 143987.8723883172
Total error: 141323.61645865036
Total error: 138777.3234009591
Total error: 136340.94553293227
Total error: 134007.167924363
Total error: 131769.32784921097
Total error: 129621.34430657266
Total error: 127557.65621879078
Total error: 125573.16812583574
Total error: 123663.20237093135
Total error: 121823.45691960417
Total error: 120049.9680779967
Total error: 118339.07748054342
Total error: 116687.40280525906
Total error: 115091.81174967671
Total error: 1135

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,6,7,6,9,5,7,7,8,4,6,...,5,6,6,9,6,7,6,7,5,4
2,5,0,8,8,7,1,7,8,10,10,...,3,1,8,0,10,1,10,0,6,10
3,6,6,6,4,8,3,4,0,10,4,...,10,3,4,10,6,10,9,4,5,3
4,5,6,6,7,4,8,7,7,9,10,...,9,8,5,9,10,10,8,6,5,9
5,10,8,3,9,0,10,3,10,10,8,...,3,10,10,10,3,3,8,10,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,5,6,6
607,7,10,3,5,10,8,3,8,9,5,...,8,5,5,5,3,5,2,8,1,0
608,6,6,5,7,6,7,6,6,7,6,...,6,6,7,6,6,6,6,6,6,5
609,2,3,9,10,6,5,10,3,9,7,...,10,1,10,9,10,5,10,3,10,10


In [8]:
# wczytujemy nazwy filmow i kategorie

movies_metadata = pandas.read_csv('ml-latest-small/movies.csv').set_index('movieId')
movies_metadata

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [9]:
# wczytujemy przykladowe grupy uzytkownikow
groups = pandas.read_csv('groups.csv').values.tolist()
groups

[[111, 307, 474, 599, 414],
 [469, 182, 232, 448, 600],
 [508, 581, 497, 402, 566],
 [300, 515, 245, 568, 507],
 [2, 371, 252, 518, 37],
 [269, 360, 469, 287, 308],
 [243, 527, 418, 118, 370],
 [186, 559, 327, 553, 314]]

In [11]:
# przygotowujemy funkcje pomocnicza

def describe_group(group, N=10):
    print(f'\n\nUser ids: {group}')
    group_size = len(group)
    
    mean_stdev = ratings.loc[group].std(axis=0).mean()
    median_stdev = ratings.loc[group].std(axis=0).median()
    std_stdev = ratings.loc[group].std(axis=0).std()
    print(f'\nMean ratings deviation: {mean_stdev}')
    print(f'Median ratings deviation: {median_stdev}')
    print(f'Standard deviation of ratings deviation: {std_stdev}')
    
    average_scores = ratings.iloc[group].mean(axis=0)
    average_scores = average_scores.sort_values()
    best_movies = [(movies_metadata['title'][movie_id], average_scores[movie_id]) for movie_id in list(average_scores[-N:].index)]
    worst_movies = [(movies_metadata['title'][movie_id], average_scores[movie_id]) for movie_id in list(average_scores[:N].index)]
    
    print('\nBest movies:')
    for movie, score in best_movies[::-1]:
        print(f'{movie}, {score}*')
    print('\nWorst movies:')
    for movie, score in worst_movies:
        print(f'{movie}, {score}*')

describe_group(groups[7])



User ids: [186, 559, 327, 553, 314]

Mean ratings deviation: 3.0869469363309743
Median ratings deviation: 3.1304951684997055
Standard deviation of ratings deviation: 0.8971171414374459

Best movies:
Far from Heaven (2002), 9.4*
Ethan Frome (1993), 9.2*
Me Myself I (2000), 9.2*
Piranha (Piranha 3D) (2010), 9.0*
Game 6 (2005), 9.0*
Crimson Rivers 2: Angels of the Apocalypse (Rivières pourpres II - Les anges de l'apocalypse, Les) (2004), 9.0*
Neon Bull (2015), 9.0*
Gangster No. 1 (2000), 9.0*
Ring of Terror (1962), 9.0*
Goodbye Girl, The (1977), 8.8*

Worst movies:
I Served the King of England (Obsluhoval jsem anglického krále) (2006), 2.2*
Sliding Doors (1998), 2.2*
Point of No Return (1993), 2.2*
Sorrow (2015), 2.4*
On the Town (1949), 2.4*
Death Wish 5: The Face of Death (1994), 2.4*
Storks (2016), 2.4*
Bittersweet Life, A (Dalkomhan insaeng) (2005), 2.4*
Ladybird Ladybird (1994), 2.4*
Josie and the Pussycats (2001), 2.4*


## Część 2. - algorytmy proste

In [12]:
# zdefiniujmy interfejs dla wszystkich algorytmow rekomendacyjnych

class Recommender:
    def recommend(self, movies, ratings, group, size):
        pass


# jako pierwszy zaimplementujemy algorytm losowy - dla porownania
    
class RandomRecommender(Recommender):
    def __init__(self):
        self.name = 'random'
        
    def recommend(self, movies, ratings, group, size):
        return sample(movies, size)

In [13]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen

class AverageRecommender(Recommender):
    def __init__(self):
        self.name = 'average'
    
    def recommend(self, movies, ratings, group, size):
        return list(ratings.iloc[group].mean(axis=0).sort_values(ascending=False).index[:size])

In [14]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen,
#   ale rownoczesnie wykluczajacy te filmy, ktore otrzymaly choc jedna ocene ponizej thresholdu

class AverageWithoutMiseryRecommender(Recommender):
    def __init__(self, score_threshold):
        self.name = 'average_without_misery'
        self.score_threshold = score_threshold
        
    def recommend(self, movies, ratings, group, size):
        return list(ratings.iloc[group].mean(axis=0).sort_values(ascending=False)[ratings.iloc[group].min(axis=0) >= self.score_threshold].index[:size])

In [ ]:
# algorytm uwzgledniajacy preferencje tylko jednego uzytkownika w kazdej iteracji

class FairnessRecommender(Recommender):
    def __init__(self):
        self.name = 'fairness'
        
    def recommend(self, movies, ratings, group, size):
        return list(ratings.iloc[choice(group)].sort_values(ascending=False).index[:size]) # check me after todo

In [ ]:
# wybrany algorytm wyborczy (dyktatura, glosowanie proste, Borda, Copeland)

class VotingRecommender(Recommender):
    def __init__(self):
        self.name = 'voting'
    
    def recommend(self, movies, ratings, group, size):
        raise NotImplementedError()

In [ ]:
# algorytm zachlanny, aproksymujacy metode Proportional Approval Voting
#   w kazdej iteracji wybieramy ten film, ktory najbardziej zwieksza zadowolenie zgodnie z punktacja PAV

class ProportionalApprovalVotingRecommender(Recommender):
    def __init__(self, threshold):
        self.threshold = threshold
        self.name = 'PAV'
        
    def recommend(self, movies, ratings, group, size):
        raise NotImplementedError()

## Część 3. - funkcje celu

In [ ]:
# dwie funkcje pomocnicze:
#  - znajdujaca ulubione filmy danego uzytkownika
#  - obliczajaca sume ocen wystawionych przez uzytkownika wszystkim filmom w rekomendacji

def top_n_movies_for_user(ratings, movies, user_id, n):
    raise NotImplementedError()

def total_score(recommendation, user_id, ratings):
    raise NotImplementedError()

In [ ]:
# funkcja obliczajaca zadowolenie pojedynczego uzytkownika
#  - iloraz zadowolenia z wygenerowanej rekomendacji oraz zadowolenia z hipotetycznej rekomendacji idealnej
def overall_user_satisfaction(recommendation, user_id, movies, ratings):
    raise NotImplementedError()

# funkcja celu - srednia z zadowolenia wszystkich uzytkownikow w grupie
def overall_group_satisfaction(recommendation, group, movies, ratings):
    raise NotImplementedError()

# funkcja celu - roznica miedzy maksymalnym i minimalnym zadowolenie w grupie
def group_disagreement(recommendation, group, movies, ratings):
    raise NotImplementedError()

## Część 4. - Sequential Hybrid Aggregation

In [ ]:
# algorytm balansujacy pomiedzy wyborem elementow o najwyzszej sredniej ocen
#   i o najwyzszej minimalnej ocenie
#   wyliczajacy w kazdej iteracji parametr alfa - jak na wykladzie
class SequentialHybridAggregationRecommender(Recommender):
     def __init__(self):
        self.name = 'sequential_hybrid_aggregation'
    
    def recommend(self, movies, ratings, group, size):
        raise NotImplementedError()

## Część 5. - porównanie algorytmów

In [ ]:
recommenders = [
    RandomRecommender(),
    AverageRecommender(),
    AverageWithoutMiseryRecommender(5),
    FairnessRecommender(),
    VotingRecommender(),
    ProportionalApprovalVotingRecommender(5),
    SequentialHybridAggregationRecommender()
]

recommendation_size = 10

# dla kazdego algorytmu:
#  - wygenerujmy jedna rekomendacje dla kazdej grupy
#  - obliczmy wartosci obu funkcji celu dla kazdej rekomendacji
#  - obliczmy srednia i odchylenie standardowe dla obu funkcji celu
#  - wypiszmy wyniki na konsole

for recommender in recommenders:
    raise NotImplementedError()